<a href="https://colab.research.google.com/github/DSGP-Group-1-EAPS/SL-Apparel-Dataset/blob/main/CatboostClassificationModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from google.colab import drive
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pandas as pd
!pip install scikit-learn==1.4.1.post1
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from datetime import timedelta
import numpy as np
!pip install catboost==1.2.3

In [25]:
pd.set_option('display.max_columns', None)

In [26]:
# Mounting the Google Drive
drive.mount('/content/drive')

training_df = pd.read_excel('/content/drive/MyDrive/DSGP_COURSEWORK/SL Apparel Dataset model/Dataset/training_dataset.xlsx')
retraining_df = pd.read_excel('/content/drive/MyDrive/DSGP_COURSEWORK/SL Apparel Dataset model/Dataset/retraining_dataset.xlsx')
sept_testing_df = pd.read_excel('/content/drive/MyDrive/DSGP_COURSEWORK/SL Apparel Dataset model/Dataset/sept_train_test.xlsx')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [61]:
retraining_df.shape

(85090, 24)

In [27]:
training_df.shape

(83236, 24)

In [28]:
training_df.head()

,Unnamed: 0,Date,Shift,Code,Department,Absenteeism Type,Status,Leave Type,Reason,Joined Date,Encoded Code,YearsWorked,DayOfWeek,LeaveMonth,LeaveYear,MainDepartment,Encoded Department,Encoded Reason,Encoded Status,Encoded Absenteeism Type,Encoded Shift,MonthlyDeptTotal,LeavesNextMonth,TargetCategory
0,0,2021-12-01,Shift A,AA369,Team - MAT 4A - BD,Informed,Notified,1.0,Maternity,2015-09-11,369,6,3,12,2021,MAT,1,10,1,0,1,127,0,A
1,1,2021-12-01,Shift A,AA362,Team - MAT 4A - BD,Informed,Notified,1.0,Maternity,2018-11-14,362,3,3,12,2021,MAT,1,10,1,0,1,127,24,B
2,2,2021-12-01,Shift A,AA359,Team - MAT 4A - BD,Informed,Notified,1.0,Maternity,2019-10-09,359,2,3,12,2021,MAT,1,10,1,0,1,127,16,B
3,3,2021-12-01,Shift A,AA541,Team - MAT 4A - BD,Informed,Notified,1.0,Maternity,2020-10-02,541,1,3,12,2021,MAT,1,10,1,0,1,127,2,A
4,4,2021-12-01,Shift A,AA398,Team - MAT 3A - BD,Informed,Notified,1.0,Maternity,2008-02-29,398,13,3,12,2021,MAT,1,10,1,0,1,127,1,A


In [29]:
training_df.tail()

,Unnamed: 0,Date,Shift,Code,Department,Absenteeism Type,Status,Leave Type,Reason,Joined Date,Encoded Code,YearsWorked,DayOfWeek,LeaveMonth,LeaveYear,MainDepartment,Encoded Department,Encoded Reason,Encoded Status,Encoded Absenteeism Type,Encoded Shift,MonthlyDeptTotal,LeavesNextMonth,TargetCategory
83231,83231,2023-07-31,Shift B,AA2877,Sewing Team - 147B - I - BD,Informed,Notified,1.0,Health Related,2021-06-29,2877,2,1,7,2023,Sewing Team,2,8,1,0,2,864,0,A
83232,83232,2023-07-31,Shift B,AA3478,Sewing Team - 121B - I - BD,Uninformed,Not Notified,1.0,No Message,2021-07-13,3478,2,1,7,2023,Sewing Team,2,13,0,1,2,864,0,A
83233,83233,2023-07-31,Shift B,AA4298,Sewing Team - 123B - I - BD,Uninformed,Not Notified,1.0,No Message,2022-03-01,4298,1,1,7,2023,Sewing Team,2,13,0,1,2,864,0,A
83234,83234,2023-07-31,Shift B,AA2830,Team - MAT 2B - BD,Informed,Notified,1.0,Maternity,2021-10-07,2830,1,1,7,2023,MAT,1,10,1,0,2,161,12,B
83235,83235,2023-07-31,Shift B,AA4275,Sewing Team - 155B - I - BD,Informed,Notified,1.0,Personal Reason,2022-10-07,4275,0,1,7,2023,Sewing Team,2,14,1,0,2,864,17,B


In [31]:
retraining_df.head()

,Unnamed: 0,Date,Shift,Code,Department,Absenteeism Type,Status,Leave Type,Reason,Joined Date,Encoded Code,YearsWorked,DayOfWeek,LeaveMonth,LeaveYear,MainDepartment,Encoded Department,Encoded Reason,Encoded Status,Encoded Absenteeism Type,Encoded Shift,MonthlyDeptTotal,LeavesNextMonth,TargetCategory
0,0,2021-12-01,Shift A,AA369,Team - MAT 4A - BD,Informed,Notified,1.0,Maternity,2015-09-11,369,6,3,12,2021,MAT,1,10,1,0,1,127,0,A
1,1,2021-12-01,Shift A,AA362,Team - MAT 4A - BD,Informed,Notified,1.0,Maternity,2018-11-14,362,3,3,12,2021,MAT,1,10,1,0,1,127,24,B
2,2,2021-12-01,Shift A,AA359,Team - MAT 4A - BD,Informed,Notified,1.0,Maternity,2019-10-09,359,2,3,12,2021,MAT,1,10,1,0,1,127,16,B
3,3,2021-12-01,Shift A,AA541,Team - MAT 4A - BD,Informed,Notified,1.0,Maternity,2020-10-02,541,1,3,12,2021,MAT,1,10,1,0,1,127,2,A
4,4,2021-12-01,Shift A,AA398,Team - MAT 3A - BD,Informed,Notified,1.0,Maternity,2008-02-29,398,13,3,12,2021,MAT,1,10,1,0,1,127,1,A


In [32]:
retraining_df.tail()

,Unnamed: 0,Date,Shift,Code,Department,Absenteeism Type,Status,Leave Type,Reason,Joined Date,Encoded Code,YearsWorked,DayOfWeek,LeaveMonth,LeaveYear,MainDepartment,Encoded Department,Encoded Reason,Encoded Status,Encoded Absenteeism Type,Encoded Shift,MonthlyDeptTotal,LeavesNextMonth,TargetCategory
85085,85085,2023-08-31,Shift A,AA3577,Sewing Team - 152A - I - BD,NaN,Not Notified,1.0,Medical Leave,2021-10-26,3577,1,4,8,2023,Sewing Team,2,11,0,2,1,852,8,B
85086,85086,2023-08-31,Shift B,AA3595,Sewing Team - 104B - I - BD,NaN,Not Notified,1.0,Personal Reason,2021-11-20,3595,1,4,8,2023,Sewing Team,2,14,0,2,2,852,1,A
85087,85087,2023-08-31,Shift A,AA3426,Sewing Team - 119A - I - BD,NaN,Not Notified,1.0,Medical Leave,2021-03-16,3426,2,4,8,2023,Sewing Team,2,11,0,2,1,852,2,A
85088,85088,2023-08-31,Shift A,AA4126,Sewing Team - 140A - I - BD,NaN,Not Notified,1.0,Health Related,2022-12-15,4126,0,4,8,2023,Sewing Team,2,8,0,2,1,852,1,A
85089,85089,2023-08-31,Shift A,AA2155,Sewing Team - 146A - II - BD,NaN,Not Notified,1.0,Personal Reason,2013-11-18,2155,9,4,8,2023,Sewing Team,2,14,0,2,1,852,6,B


In [33]:
sept_testing_df.shape

(2317, 13)

In [34]:
sept_testing_df.head()

,Unnamed: 0,Encoded Code,Encoded Department,YearsWorked,DayOfWeek,LeaveMonth,LeaveYear,Encoded Reason,Encoded Status,Encoded Absenteeism Type,Encoded Shift,MonthlyDeptTotal,TargetCategory
0,98398,365,2,24,5,9,2023,15,1,0,1,1003,A
1,98399,2278,1,14,5,9,2023,10,1,0,1,133,B
2,98400,393,1,13,5,9,2023,10,1,0,1,133,B
3,98402,4102,1,1,5,9,2023,10,1,0,1,133,B
4,98403,1442,2,9,5,9,2023,10,1,0,1,1003,B


In [35]:
sept_testing_df.tail()

,Unnamed: 0,Encoded Code,Encoded Department,YearsWorked,DayOfWeek,LeaveMonth,LeaveYear,Encoded Reason,Encoded Status,Encoded Absenteeism Type,Encoded Shift,MonthlyDeptTotal,TargetCategory
2312,103784,4250,2,0,4,9,2023,14,1,0,1,1003,B
2313,103785,4278,2,1,4,9,2023,14,1,0,1,1003,A
2314,103786,3950,2,1,5,9,2023,14,1,0,1,1003,A
2315,103787,1143,2,8,5,9,2023,14,1,0,1,1003,A
2316,103788,3922,2,1,5,9,2023,14,1,0,1,1003,B


In [36]:
features  = ['Encoded Code', 'Encoded Department', 'YearsWorked', 'DayOfWeek',
             'LeaveMonth', 'LeaveYear', 'Encoded Reason','Encoded Status',
             'Encoded Absenteeism Type', 'Encoded Shift', 'MonthlyDeptTotal']

In [37]:
X_train = training_df[features]
Y_train = training_df['TargetCategory']

In [38]:
X_retrain = retraining_df[features]
Y_retrain = retraining_df['TargetCategory']

In [39]:
X_retrain.shape

(85090, 11)

In [40]:
X_test = sept_testing_df[features]
Y_test = sept_testing_df['TargetCategory']

In [41]:
X_test.head()

,Encoded Code,Encoded Department,YearsWorked,DayOfWeek,LeaveMonth,LeaveYear,Encoded Reason,Encoded Status,Encoded Absenteeism Type,Encoded Shift,MonthlyDeptTotal
0,365,2,24,5,9,2023,15,1,0,1,1003
1,2278,1,14,5,9,2023,10,1,0,1,133
2,393,1,13,5,9,2023,10,1,0,1,133
3,4102,1,1,5,9,2023,10,1,0,1,133
4,1442,2,9,5,9,2023,10,1,0,1,1003


#Model Training

In [65]:
from catboost import CatBoostClassifier

# Define and train the CatBoost model with proper parameters
model_catboost = CatBoostClassifier(iterations=1500, learning_rate=0.05, depth=8, loss_function='Logloss')
model_catboost.fit(X_train, Y_train)

0:	learn: 0.6796015	total: 28ms	remaining: 41.9s
1:	learn: 0.6673717	total: 53.8ms	remaining: 40.3s
2:	learn: 0.6567990	total: 81.2ms	remaining: 40.5s
3:	learn: 0.6479296	total: 107ms	remaining: 39.8s
4:	learn: 0.6396683	total: 134ms	remaining: 40.1s
5:	learn: 0.6320246	total: 164ms	remaining: 40.9s
6:	learn: 0.6253762	total: 190ms	remaining: 40.6s
7:	learn: 0.6195059	total: 236ms	remaining: 44.1s
8:	learn: 0.6129768	total: 262ms	remaining: 43.4s
9:	learn: 0.6071626	total: 287ms	remaining: 42.8s
10:	learn: 0.6020828	total: 318ms	remaining: 43s
11:	learn: 0.5970512	total: 352ms	remaining: 43.6s
12:	learn: 0.5927403	total: 401ms	remaining: 45.8s
13:	learn: 0.5889488	total: 463ms	remaining: 49.2s
14:	learn: 0.5859195	total: 535ms	remaining: 53s
15:	learn: 0.5825087	total: 563ms	remaining: 52.2s
16:	learn: 0.5786769	total: 599ms	remaining: 52.3s
17:	learn: 0.5756390	total: 656ms	remaining: 54s
18:	learn: 0.5724004	total: 702ms	remaining: 54.7s
19:	learn: 0.5700587	total: 729ms	remaining: 5

In [64]:
from catboost import CatBoostClassifier, Pool, cv
from sklearn.model_selection import GridSearchCV

# Define the CatBoost model
model_catboost = CatBoostClassifier(loss_function='Logloss')

# Define the parameter grid
param_grid = {
    'iterations': [500, 1000, 1500],
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [4, 6, 8]
}

# Perform grid search
grid_search = GridSearchCV(estimator=model_catboost, param_grid=param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_retrain, Y_retrain)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)


Streaming output truncated to the last 5000 lines.
1002:	learn: 0.1211621	total: 19.3s	remaining: 9.55s
1003:	learn: 0.1210210	total: 19.3s	remaining: 9.53s
1004:	learn: 0.1209521	total: 19.3s	remaining: 9.51s
1005:	learn: 0.1208611	total: 19.3s	remaining: 9.49s
1006:	learn: 0.1207758	total: 19.3s	remaining: 9.47s
1007:	learn: 0.1207071	total: 19.4s	remaining: 9.45s
1008:	learn: 0.1206634	total: 19.4s	remaining: 9.43s
1009:	learn: 0.1206078	total: 19.4s	remaining: 9.41s
1010:	learn: 0.1205211	total: 19.4s	remaining: 9.38s
1011:	learn: 0.1204702	total: 19.4s	remaining: 9.36s
1012:	learn: 0.1204010	total: 19.4s	remaining: 9.34s
1013:	learn: 0.1202182	total: 19.5s	remaining: 9.32s
1014:	learn: 0.1201285	total: 19.5s	remaining: 9.3s
1015:	learn: 0.1200842	total: 19.5s	remaining: 9.28s
1016:	learn: 0.1200209	total: 19.5s	remaining: 9.26s
1017:	learn: 0.1199657	total: 19.5s	remaining: 9.24s
1018:	learn: 0.1198628	total: 19.5s	remaining: 9.22s
1019:	learn: 0.1198264	total: 19.6s	remaining: 9.

In [66]:
from sklearn.metrics import accuracy_score

# Get predictions on the test set

model_catboost.fit(X_retrain,Y_retrain)


0:	learn: 0.6797606	total: 24.6ms	remaining: 36.8s
1:	learn: 0.6675074	total: 47ms	remaining: 35.2s
2:	learn: 0.6566940	total: 69.6ms	remaining: 34.7s
3:	learn: 0.6478441	total: 90.8ms	remaining: 33.9s
4:	learn: 0.6397042	total: 112ms	remaining: 33.6s
5:	learn: 0.6315973	total: 135ms	remaining: 33.6s
6:	learn: 0.6246287	total: 157ms	remaining: 33.5s
7:	learn: 0.6191687	total: 179ms	remaining: 33.5s
8:	learn: 0.6139857	total: 201ms	remaining: 33.4s
9:	learn: 0.6079588	total: 224ms	remaining: 33.3s
10:	learn: 0.6026709	total: 249ms	remaining: 33.7s
11:	learn: 0.5974966	total: 271ms	remaining: 33.6s
12:	learn: 0.5928228	total: 296ms	remaining: 33.9s
13:	learn: 0.5886899	total: 318ms	remaining: 33.8s
14:	learn: 0.5849688	total: 346ms	remaining: 34.2s
15:	learn: 0.5810332	total: 370ms	remaining: 34.3s
16:	learn: 0.5779287	total: 394ms	remaining: 34.4s
17:	learn: 0.5752566	total: 419ms	remaining: 34.5s
18:	learn: 0.5725891	total: 447ms	remaining: 34.9s
19:	learn: 0.5698727	total: 472ms	remai

check accuracy

In [67]:
# Get predictions on the test set
predictions_catboost = model_catboost.predict(X_test)

# Calculate accuracy on the test set
accuracy_catboost = accuracy_score(Y_test, predictions_catboost)
print("CatBoost Accuracy:", accuracy_catboost)

CatBoost Accuracy: 0.8208890807078119


Classfication report

In [68]:
from sklearn.metrics import classification_report

# Get predictions on the test set
predictions_catboost = model_catboost.predict(X_test)

# Generate classification report
report = classification_report(Y_test, predictions_catboost)

# Print the classification report
print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

           A       0.81      0.93      0.87      1439
           B       0.85      0.64      0.73       878

    accuracy                           0.82      2317
   macro avg       0.83      0.78      0.80      2317
weighted avg       0.83      0.82      0.81      2317



saving the model

In [69]:
import pickle
# Save the model
with open('/content/drive/MyDrive/DSGP_COURSEWORK/SL Apparel Dataset model/Models/Catboost_model_original.pkl', 'wb') as file:
    pickle.dump(model_catboost, file)